In [10]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '문서 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            doc_id = line['doc_id']
            doc_type = line['doc_type']
            title = line['title']
            ipc = line['ipc']
            summary_entire = line.get('summary_entire', [])
            summary_section = line.get('summary_section', [])
            #### data preprocess end 
            for summary in summary_entire + summary_section:
                #### LM(한국어)
                data = {'text': None}
                ## preprocess data from line
                text = '##doc_type : ' + doc_type + '\n'
                text += '##title : ' + title + '\n'
                text += '##text : ' + summary['orginal_text'] + '\n'
                data['text'] = text
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(한국어) end


                #### 문서 요약
                data = {'input': {}, 'output': None}
                ## preprocess data from line
                data['input']['doc_id'] = doc_id
                data['input']['doc_type'] = doc_type
                data['input']['ipc'] = ipc
                data['input']['title'] = title
                data['input']['text'] = summary['orginal_text']
                data['output'] = summary['summary_text']
                ## preprocess data from line end
                task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 문서 요약 end

    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 3/3 [00:47<00:00, 15.84s/it]
